کتابخانه های مورد نیاز

دانلود دیتاست مورد استفاده:   https://www.mediafire.com/file/j03djpy140rno4d/data.rar/file

In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

from PIL import Image 
import random
import tensorflow as tf  


به جهت ثابت نگه داشتن دقت

In [2]:
seed_value = 42
np.random.seed(seed_value)  
random.seed(seed_value)  
tf.random.set_seed(seed_value) 

وارد کردن دیتاست

In [3]:
data_dir = "data"  
categories = os.listdir(data_dir) 

داده‌ها و برچسب‌ها 

In [4]:
images = []
labels = []

فرمت‌های تصویری پشتیبانی‌شده

In [5]:
valid_formats = ('.jpg', '.png', '.gif', '.jpeg')

تعداد تصاویر در هر پوشه

In [6]:
for category in categories:  
    category_path = os.path.join(data_dir, category)
    image_count = len([file for file in os.listdir(category_path) if file.lower().endswith(valid_formats)])
    print(f"Number_Images_'{category}' : {image_count} images.")


Number_Images_'Angry' : 86 images.
Number_Images_'Happy' : 100 images.
Number_Images_'Sad' : 79 images.


تعداد فرمت های تصاویر برای هر احساس

In [7]:
for category in categories:
    category_path = os.path.join(data_dir, category)
    format_count = {ext: 0 for ext in valid_formats}
    
    for file in os.listdir(category_path):
        file_lower = file.lower()
        for ext in valid_formats:
            if file_lower.endswith(ext):
                format_count[ext] += 1

    print(f"Emotion: '{category}'")
    for ext, count in format_count.items():
        print(f"  {ext}:  {count}")


Emotion: 'Angry'
  .jpg:  67
  .png:  14
  .gif:  0
  .jpeg:  5
Emotion: 'Happy'
  .jpg:  80
  .png:  8
  .gif:  0
  .jpeg:  12
Emotion: 'Sad'
  .jpg:  69
  .png:  4
  .gif:  1
  .jpeg:  5


تغییر سایز

In [8]:
img_size = 32

بارگذاری و پردازش تصاویر

In [9]:
for label, category in enumerate(categories):  
    category_path = os.path.join(data_dir, category)  
    for file in os.listdir(category_path):  
        if file.lower().endswith(valid_formats):  
            file_path = os.path.join(category_path, file)  
            try:  
                if file.lower().endswith('.gif'):  
                    img = Image.open(file_path)  # بارگذاری GIFها با استفاده از Pillow  
                    img = img.convert('RGB')  # تبدیل به فرمت RGB  
                    img_resized = img.resize((img_size, img_size), Image.LANCZOS)  # تغییر اندازه با آنتی‌الیزینگ  
                    img_resized = cv2.cvtColor(np.array(img_resized), cv2.COLOR_RGB2BGR)  # تبدیل به فرمت OpenCV  
                else:  
                    img = cv2.imread(file_path, cv2.IMREAD_COLOR)  # بارگذاری سایر تصاویر با OpenCV  
                    if img is not None:  
                        img_resized = cv2.resize(img, (img_size, img_size))  

                images.append(img_resized)  # اضافه کردن تصویر تغییر اندازه داده شده  
                labels.append(label)          # اضافه کردن برچسب  
            except Exception as e:  
                print(f"Error processing {file_path}: {e}")

 و نرمال سازی تبدیل به آرایه numpy 

In [10]:
images = np.array(images) / 255.0  
labels = np.array(labels)

ورود لیبل ها به صورت one hot

In [11]:
labels = to_categorical(labels, num_classes=len(categories))

تقسیم داده‌های آموزشی و آزمایشی

In [12]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

تغییر شکل تصاویر برای ورود به RNN

In [13]:
X_train = X_train.reshape(X_train.shape[0], 1, img_size * img_size * 3)  
X_test = X_test.reshape(X_test.shape[0], 1, img_size * img_size * 3)

In [14]:
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

Shape of X_train: (212, 1, 3072)
Shape of y_train: (212, 3)


یکسان سازی شکل 

In [15]:
y_train = y_train.reshape(y_train.shape[0], -1)
y_test = y_test.reshape(y_test.shape[0], -1)

ساخت مدل RNN_GRU

In [16]:
model = Sequential()
model.add(GRU(128, input_shape=(1, img_size * img_size * 3), return_sequences=False))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(len(categories), activation='softmax'))


c:\Users\fatem\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


کامپایل مدل

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

حذف بعد اضافی از y_train و y_test

In [18]:
y_train = y_train.reshape(y_train.shape[0], -1)
y_test = y_test.reshape(y_test.shape[0], -1)

print(f"y_train shape after reshape: {y_train.shape}")

y_train shape after reshape: (212, 3)


آموزش مدل

In [19]:
lr_scheduler = ReduceLROnPlateau( monitor='val_loss', factor=0.5, patience=5, verbose=1 )  
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=90, batch_size=32, callbacks=[lr_scheduler])

Epoch 1/90
7/7 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.2610 - loss: 1.7195 - val_accuracy: 0.3962 - val_loss: 1.2648 - learning_rate: 0.0010
Epoch 2/90
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 210ms/step - accuracy: 0.3509 - loss: 1.1862 - val_accuracy: 0.3774 - val_loss: 1.1209 - learning_rate: 0.0010
Epoch 3/90
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 231ms/step - accuracy: 0.4138 - loss: 1.1926 - val_accuracy: 0.3774 - val_loss: 1.0880 - learning_rate: 0.0010
Epoch 4/90
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 280ms/step - accuracy: 0.3053 - loss: 1.2040 - val_accuracy: 0.3585 - val_loss: 1.0957 - learning_rate: 0.0010
Epoch 5/90
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 208ms/step - accuracy: 0.3670 - loss: 1.1394 - val_accuracy: 0.3585 - val_loss: 1.1073 - learning_rate: 0.0010
Epoch 6/90
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 185ms/step - accuracy: 0.3854 - loss: 1.1026 - val_accuracy: 0.2453 - val_loss: 1.1089 - learning_rate: 0.0010
Epoch 7/90
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 0.3586 - loss: 1.1367 - val_accuracy: 0.

ارزیابی مدل

In [20]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.6590 - loss: 0.9188 
Test Accuracy: 66.04%


رسم نمودار

In [ ]:
# accuracy  
plt.figure(figsize=(12, 5))  
plt.subplot(1, 2, 1)  
plt.plot(history.history['accuracy'], label='Training Accuracy')  
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')  
plt.title('Model Accuracy')  
plt.xlabel('Epochs')  
plt.ylabel('Accuracy')  
plt.legend(loc='lower right')  

# loss  
plt.subplot(1, 2, 2)  
plt.plot(history.history['loss'], label='Training Loss')  
plt.plot(history.history['val_loss'], label='Validation Loss')  
plt.title('Model Loss')  
plt.xlabel('Epochs')  
plt.ylabel('Loss')  
plt.legend(loc='upper right')  

#------------------------------------------------------------------

plt.tight_layout()  
plt.show()